# Lesson 07 Reference

# Handcalcs

## Use handcalcs in a notebook cell with `%%render`

**Typical import**

```python
import handcalcs.render
```

The `%%render` "cell magic" only has meaning within Jupyterlab notebook cell. If you were to use it in a `.py` python file, it would cause an error because it has no meaning. 

Example:

**Cell 1**
```python
%%render params
DL = 2.5 # kPa
LL = 4.8 # kPa
SL = 1.0 # kPa
```

**Cell 2**
```python
%%render
FL_LC3a = 1.25*DL + 1.5*LL + 1.0*SL # kPa
```

## Use handcalcs in functions with the `@handcalc` decorator

**Typical import**
```python
from handcalcs.decorator import handcalc
```

While the `%%render` cell magic can only be used in Jupyter and not general Python, you can still use handcalcs in general Python by using the `@handcalc` decorator. It works like this:

```python
@handcalc(jupyter_display=True)
def LC3a(DL: float, LL: float, SL: float) -> float:
    """NBCC Loadcase 3a"""
    FL = 1.25*DL + 1.5*LL + 1.0*SL # kPa
    return FL
```

Similar to a cell magic, a decorator is a simply a function that gets called on your function before your function executes.

When the function executes, you will see the code within your function get rendered in your notebook AND you will also get your function return value.

**Usage**

```python
DL = 2.5
LL = 4.8
SL = 1.0
factored = LC3a(DL, LL, SL)
print(factored)
```

Now your function will only return one value, your return value, and the _function code_ will be rendered in the cell.

In [14]:
from handcalcs.decorator import handcalc

In [16]:
@handcalc(jupyter_display=True)
def LC3a(DL: float, LL: float, SL: float) -> float:
    """NBCC Loadcase 3a"""
    FL = 1.25*DL + 1.5*LL + 1.0*SL # kPa
    return FL

In [18]:
DL = 2.5
LL = 4.8
SL = 1.0
factored = LC3a(DL, LL, SL)
print(factored)

<IPython.core.display.Latex object>

11.325


## How it works

1. When you run your cell, handcalcs first reads the code in your cell.
2. handcalcs tells Jupyter to run the code in the cell and all values are calculated and assigned to the appropriate variables.
3. It splits the numbers and operators apart and then uses several `for` loops to _transform_ each of the python code strings into $\LaTeX$ strings. 
4. Finally, it gets the calculated results from Python and puts the appropriate result at the end of your calculation code.


**For example:**

If your code cell looked like this:

```python
a = 4
b = 5.2
c = 2 * a + b
```

Then the cell source code looks like this:

```python
source_code = 'a = 4\nb = 5.2\nc = 2 * a + b'
```

In [13]:
source_code = 'a = 4\nb = 5.2\nc = 2 * a + b'

Now, to split up the code into individual lines and then individual pieces of arithmetic:
```python
# Split the cell code into lines
lines_of_code = source_code.split("\n")

# Split up the lines of code into atomic pieces
split_lines = []
for line_of_code in lines_of_code:
    split_lines.append(line_of_code.split(" "))
```

In [ ]:
# Split the cell code into lines
lines_of_code = source_code.split("\n")

# Split up the lines of code into atomic pieces
split_lines = []
for line_of_code in lines_of_code:
    split_lines.append(line_of_code.split(" "))

From the Jupyter back-end, we can get a dictionary with all of the variable values which result from running the cell:

```python
results = {"a": 4, "b": 5.2, "c": 13.2}
```


In [18]:
results = {"a": 4, "b": 5.2, "c": 13.2}

The first two lines just have variable assignments so we will only update the information in the last line.

The idea is to show the formula, the numeric substitution, and then the result. So, lets change the form of the third line so it looks more like symbol, formula, numeric susbtitution, result.

```python
last_line = split_lines[2]
result_symbol = last_line[0]
formula = last_line[2:]

numeric_substitution = []
for symbol in formula:
    numeric_substitution.append(results.get(symbol, symbol))

result_value = results.get(result_symbol, result_symbol)

new_line = [result_symbol] + ["="] + formula + ["="] + numeric_substitution + ["="] + [result_value]

split_lines[2] = new_line
```

Now that we have all of the pieces, put each line back together as strings but keep them as separate lines.

```python
converted_lines = []
for line_as_list in split_lines:
    all_elements_as_strings = [str(element) for element in line_as_list]
    line_as_string = " ".join(all_elements_as_strings)
    converted_lines.append(line_as_string)
```

And last, we will wrap these strings in a "Latex" environment and put Latex line breaks around them. Latex is a different programming language for displaying math symbols.

```python
latex_start =  "\[\n\\begin{aligned}\n"
latex_end = "\\end{aligned}\n\]"
line_break_symbol = "\\\\[10pt]\n"

joined_lines_as_latex = line_break_symbol.join(converted_lines)

final_code = latex_start + joined_lines_as_latex + latex_end
```
    

Last, use the special `IPython.display` functions to render the Latex code in the browser.

```python
from IPython.display import Latex, display_latex

latex_code = Latex(final_code)
display_latex(latex_code)
```